##### Thank you very much to Steven Morse for writing his article about Spotify for Python. This is a great resource for anyone who wants to get started. The code below is a simple copy of Steven's work and what he offers is a great resource to improve your understanding of the process. I would say ahead of time that I have taken a Pandas heavy approach to the problem which is differnt to Steven. That being said the API POSTS and GETS were Steven's work! I am sure Steven has his own references :) 
##### Continuing on, we all advocate for what we think is the best approach but really what I am saying is that I understand Pandas quite well and if can write the code in one line instead of 10 I will. :)
##### Steven has a great pure Python approach which I think is really interesting 
##### Defintely check his article out!  
##### Exploring the Spotify API in Python - Steven Morse
##### https://stmorse.github.io/journal/spotify-api.html

In [2]:
import requests
from lookup import *
import pandas as pd

In [6]:
BASE_URL = 'https://api.spotify.com/v1/'

In [7]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [8]:
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

##### So here is where the fun begins :) I am a big John Mayer fan. 

Why? 

I'm not sure to be honest, he in many ways is the opposite of what I typically seek out. He can come across as inauthentic and clichéd. That being said he is, to this day, the reason I still play guitar... And his songs still surprise me. He isn't a great singer, his lyrics are cheesy, but his guitar playing is somewhat reknowned and his guitar playing is a major hook to many people.

Much a like an observer during a magic show, I am here for John Mayer and spectical of the show. If I'm honest, I can forget about the rehersed timestamps and cues ... if there are any...

The reason I am here is to see if the data can explain my puppy dog sickness for John Mayer and his music.

In [9]:
# John Mayer Albums
artist_id = '0hEurMDQu99nJRq8pTxO14'

# pull all artists albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

In [198]:
d.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

##### Generate album information

In [12]:
albumInfo = pd.DataFrame.from_dict(d['items']).drop(['album_type','album_group','type'],axis=1)

# Convert to datetime
albumInfo['release_date'] = pd.to_datetime(albumInfo['release_date'])

# Clean from list like format to value 
albumInfo['external_urls'] = albumInfo['external_urls'].apply(lambda x: x['spotify'])

# Clean from dictionary format to value
albumInfo['images'] = albumInfo['images'].apply(lambda x: x[0]['url'])

albumInfo.to_csv('song_data_output/album_info.csv')

##### Find songs for each album

In [14]:
songInfoCommbined = pd.DataFrame()
for albumId,albumName in zip(albumInfo['id'],albumInfo['name']):
    print(albumName)
    
    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + albumId + '/tracks', headers=headers)
    tracks = r.json()['items']
    songInfo = pd.DataFrame.from_dict(tracks)
    
    # Manually add album name
    songInfo['album_name'] = str(albumName)
    
    songInfoCommbined = pd.concat([songInfoCommbined,songInfo],axis=0)

songInfoCommbined['id'].reset_index(drop=True,inplace=True)
songInfoCommbined.to_csv('song_data_output/song_info_combined.csv')

Sob Rock
The Search for Everything
Paradise Valley
Born and Raised
Battle Studies
Where the Light Is: John Mayer Live In Los Angeles
Continuum
As/Is - Live
As/Is: Live In Cleveland/Cincinnati, OH - 8/03-8/04/04
As/Is: Houston, TX - 7/24/04
As/Is: Live In Philadelphia, PA/Hartford, CT - 8/14-8/15/04
As/Is: Live In Houston, TX - 7/24/04
As/Is: Live In Mountain View, CA - 7/16/04
Heavier Things
Any Given Thursday
Room For Squares
Inside Wants Out


##### Find audio features for each song

In [327]:
songAudioFeaturesCombined = pd.DataFrame()

# John Mayer Songs Information
for i in songInfoCommbined['id']:
    f = requests.get(BASE_URL + 'audio-features/' + i,headers=headers)
    f = f.json()
    f = f.items()
    
    # Reshape vertical Series to horizontal dataframe
    df = pd.DataFrame.from_dict(f).transpose()
    
    #set column names equal to values in row index position 0
    df.columns = df.iloc[0]
    
    #remove first row from DataFrame
    df = df[1:]
    
    #concat updated DataFrame
    songAudioFeaturesCombined = pd.concat([songAudioFeaturesCombined,df],axis=0)

songAudioFeaturesCombined.reset_index(drop=True,inplace=True)
songAudioFeaturesCombined.to_csv('song_data_output/song_audio_features_combined.csv')